<center>
<img src="https://habrastorage.org/files/fd4/502/43d/fd450243dd604b81b9713213a247aa20.jpg" />
</center> 
     
## <center>  [mlcourse.ai](https://mlcourse.ai) – Open Machine Learning Course 

#### <center> Author: [Yury Kashnitsky](https://yorko.github.io) (@yorko) 

# <center>Assignment #2. Fall 2019
## <center> Part 2. Gradient boosting

**In this assignment, you're asked to beat a baseline in the ["Flight delays" competition](https://www.kaggle.com/c/flight-delays-fall-2018).**

This time we decided to share a pretty decent CatBoost baseline, you'll have to improve the provided solution.

Prior to working on the assignment, you'd better check out the corresponding course material:
 1. [Classification, Decision Trees and k Nearest Neighbors](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic03_decision_trees_kNN/topic3_decision_trees_kNN.ipynb?flush_cache=true), the same as an interactive web-based [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-3-decision-trees-and-knn) 
 2. Ensembles:
  - [Bagging](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic05_ensembles_random_forests/topic5_part1_bagging.ipynb?flush_cache=true), the same as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-5-ensembles-part-1-bagging)
  - [Random Forest](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic05_ensembles_random_forests/topic5_part2_random_forest.ipynb?flush_cache=true), the same as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-5-ensembles-part-2-random-forest)
  - [Feature Importance](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic05_ensembles_random_forests/topic5_part3_feature_importance.ipynb?flush_cache=true), the same as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-5-ensembles-part-3-feature-importance)
 3. - [Gradient boosting](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic10_boosting/topic10_gradient_boosting.ipynb?flush_cache=true), the same as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-10-gradient-boosting) 
   - Logistic regression, Random Forest, and LightGBM in the "Kaggle Forest Cover Type Prediction" competition: [Kernel](https://www.kaggle.com/kashnitsky/topic-10-practice-with-logit-rf-and-lightgbm) 
 4. You can also practice with demo assignments, which are simpler and already shared with solutions:
  - "Decision trees with a toy task and the UCI Adult dataset": [assignment](https://www.kaggle.com/kashnitsky/a3-demo-decision-trees) + [solution](https://www.kaggle.com/kashnitsky/a3-demo-decision-trees-solution)
  - "Logistic Regression and Random Forest in the credit scoring problem": [assignment](https://www.kaggle.com/kashnitsky/assignment-5-logit-and-rf-for-credit-scoring) + [solution](https://www.kaggle.com/kashnitsky/a5-demo-logit-and-rf-for-credit-scoring-sol)
 5. There are also 7 video lectures on trees, forests, boosting and their applications: [mlcourse.ai/video](https://mlcourse.ai/video) 
 6. mlcourse.ai tutorials on [categorical feature encoding](https://www.kaggle.com/waydeherman/tutorial-categorical-encoding) (by Wayde Herman) and [CatBoost](https://www.kaggle.com/mitribunskiy/tutorial-catboost-overview) (by Mikhail Tribunskiy)
 7. Last but not the least: [Public Kernels](https://www.kaggle.com/c/flight-delays-fall-2018/notebooks) in this competition

### Your task is to:
 1. beat **"A2 baseline (10 credits)"** on Public LB (**0.75914** LB score)
 2. rename your [team](https://www.kaggle.com/c/flight-delays-fall-2018/team) in full accordance with A1 and the [course rating](https://docs.google.com/spreadsheets/d/15e1K0tg5ponA5R6YQkZfihrShTDLAKf5qeKaoVCiuhQ/) (to appear on 16.09.2019)
 
This task is intended to be relatively easy. Here you are not required to upload your reproducible solution.
 
### <center> Deadline for A2: 2019 October 6, 20:59 CET (London time)

In [ ]:
import warnings
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import roc_auc_score
from catboost import Pool, CatBoostClassifier, cv
import matplotlib.pyplot as plt
import lightgbm as lgb
import seaborn as sns

import matplotlib.pyplot as plt

from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt 

**Read the data**

In [ ]:
PATH_TO_DATA = Path('../input/flight-delays-fall-2018/')

In [ ]:
train_df = pd.read_csv(PATH_TO_DATA / 'flight_delays_train.csv')

In [ ]:
train_df.head(15)

In [ ]:
test_df = pd.read_csv(PATH_TO_DATA / 'flight_delays_test.csv')

In [ ]:
test_df.head()

**Create only one feature - “flight” (this you need to improve - add more features)**

In [ ]:
train_df['flight'] = train_df['Origin'] + '=>' + train_df['Dest']
test_df['flight'] = test_df['Origin'] + '=>' + test_df['Dest']

train_df['flight_back'] = train_df['Dest'] + '<=' + train_df['Origin']
test_df['flight_back'] = test_df['Dest'] + '<=' + test_df['Origin']
train_df['flight_carrier'] = train_df['UniqueCarrier'] + '=' + train_df['flight']
test_df['flight_carrier'] = test_df['UniqueCarrier'] + '=' + test_df['flight']

"""for col in ['UniqueCarrier', 'Origin', 'Dest', 'flight', 'flight_back', 'flight_carrier']:
    train_df[col] = pd.factorize(train_df[col])[0]
for col in ['UniqueCarrier', 'Dest', 'Origin', 'flight', 'flight_back', 'flight_carrier']:
    test_df[col] = pd.factorize(test_df[col])[0]"""

In [ ]:
#Add DepHour
train_df['DepHour'] = train_df['DepTime'] // 100
#add DepMin
train_df['Dep_minute'] =  train_df['DepTime']%100
train_df['DepHour'].replace(to_replace=[24,25], value=0, inplace=True)

#Add DepHour + minute
test_df['DepHour'] = test_df['DepTime'] // 100
test_df['Dep_minute'] =  test_df['DepTime']%100
test_df['DepHour'].replace(to_replace=[24,25], value=0, inplace=True)

In [ ]:
# Daytime
"""train_df['daytime'] = pd.cut(train_df['DepHour'], bins=[0, 6, 12, 18, 23], include_lowest=True).astype('object')
test_df['daytime'] = pd.cut(test_df['DepHour'], bins=[0, 6, 12, 18, 23], include_lowest=True).astype('object')

for col in ['daytime']:
    train_df[col] = pd.factorize(train_df[col])[0]
for col in ['daytime']:
    test_df[col] = pd.factorize(test_df[col])[0]"""
    


In [ ]:
train_df.head()

In [ ]:
test_df['s_hour'] = np.sin(2*np.pi*test_df['DepHour']/24)
test_df['c_hour'] = np.cos(2*np.pi*test_df['DepHour']/24)

train_df['s_hour'] = np.sin(2*np.pi*train_df['DepHour']/24)
train_df['c_hour'] = np.cos(2*np.pi*train_df['DepHour']/24)

train_df['Dep_hour_flag'] = ((train_df['DepHour'] >= 6) & (train_df['DepHour'] < 23)).astype(np.int64)
test_df['Dep_hour_flag'] = ((test_df['DepHour'] >= 6) & (test_df['DepHour'] < 23)).astype(np.int64)

In [ ]:
train_df['summer'] = (train_df['Month'].isin(['c-6', 'c-7', 'c-8'])).astype(np.int64)
train_df['autumn'] = (train_df['Month'].isin(['c-9', 'c-10', 'c-11'])).astype(np.int64)
train_df['winter'] = (train_df['Month'].isin(['c-12', 'c-1', 'c-2'])).astype(np.int64)
train_df['spring'] = (train_df['Month'].isin(['c-3', 'c-4', 'c-5'])).astype(np.int64)

test_df['summer'] = (test_df['Month'].isin(['c-6', 'c-7', 'c-8'])).astype(np.int64)
test_df['autumn'] = (test_df['Month'].isin(['c-9', 'c-10', 'c-11'])).astype(np.int64)
test_df['winter'] = (test_df['Month'].isin(['c-12', 'c-1', 'c-2'])).astype(np.int64)
test_df['spring'] = (test_df['Month'].isin(['c-3', 'c-4', 'c-5'])).astype(np.int64)

In [ ]:
#train_df['delay_time'] = ((train_df['DepHour'] >= 13) & (train_df['DepHour'] < 24) | (train_df['DepHour'] < 5)).astype('int64')
#train_df['other_time'] = ((train_df['DepHour'] < 9) & (train_df['DepHour'] >=5)).astype('object')
#train_df['middle_time'] = ((train_df['DepHour'] >= 9) & (train_df['DepHour'] < 13)).astype('object')

In [ ]:
#test_df['delay_time'] = ((test_df['DepHour'] >= 13) & (test_df['DepHour'] < 24) | (test_df['DepHour'] < 5)).astype('int64')
#test_df['other_time'] = ((test_df['DepHour'] < 9) & (test_df['DepHour'] >=5)).astype('object')
#test_df['middle_time'] = ((test_df['DepHour'] >= 9) & (test_df['DepHour'] < 13)).astype('object')

In [ ]:
"""train_df['morning'] = ((train_df['DepHour'] >= 6) & (train_df['DepHour'] < 12)).astype(np.int32)
train_df['day'] = ((train_df['DepHour'] >= 12) & (train_df['DepHour'] < 18)).astype(np.int32)
train_df['evening'] = ((train_df['DepHour'] >= 18) & (train_df['DepHour'] < 24)).astype(np.int32)
train_df['night'] = ((train_df['DepHour'] >= 0) & (train_df['DepHour'] < 6)).astype(np.int32)
train_df['low_delay'] = ((train_df['DepHour'] >= 4) & (train_df['DepHour'] < 9)).astype(np.int32)
#train_df['high_delay'] = ((train_df['DepHour'] >= 4) & (train_df['DepHour'] < 9)).astype(np.int32)

test_df['morning'] = ((test_df['DepHour'] >= 6) & (test_df['DepHour'] < 12)).astype(np.int32)
test_df['day'] = ((test_df['DepHour'] >= 12) & (test_df['DepHour'] < 18)).astype(np.int32)
test_df['evening'] = ((test_df['DepHour'] >= 18) & (test_df['DepHour'] < 24)).astype(np.int32)
test_df['night'] = ((test_df['DepHour'] >= 0) & (test_df['DepHour'] < 6)).astype(np.int32)
test_df['low_delay'] = ((test_df['DepHour'] >= 4) & (test_df['DepHour'] < 9)).astype(np.int32)
#test_df['high_delay'] = ((test_df['DepHour'] >= 4) & (test_df['DepHour'] < 9)).astype(np.int32)"""

In [ ]:
"""train_df['month_x'] = ((train_df['Month'].str[2:].isin([12, 6, 7]))).astype('int')
test_df['month_x'] = ((test_df['Month'].str[2:].isin([12, 1]))).astype('int')

train_df['month_y'] = ((train_df['Month'].str[2:].isin([4, 5, 9, 2]))).astype('int')
test_df['month_y'] = ((test_df['Month'].str[2:].isin([12, 1]))).astype('int')"""
#train_df['winter'] = ((train_df['Month'].isin([12, 1, 2]))).astype('int')
#test_df['winter'] = ((test_df['Month'].isin([12, 1, 2]))).astype('int')

#train_df['spring'] = ((train_df['Month'].isin([3, 4, 5]))).astype('int')
#test_df['spring'] = ((test_df['Month'].isin([3, 4, 5]))).astype('int')

#train_df['summer'] = ((train_df['Month'].isin([6, 7, 8]))).astype('int')
#test_df['summer'] = ((test_df['Month'].isin([6, 7, 8]))).astype('int')

"""train_df['autumn'] = ((train_df['Month'].isin([9, 10, 11]))).astype('int')
test_df['autumn'] = ((test_df['Month'].isin([9, 10, 11]))).astype('int')"""
"""train_df['week_high'] = ((df_train['DayOfWeek'].isin([4, 5, 1, 7]))).astype('int')
test_df['week_high'] = ((test_df['DayOfWeek'].isin([12, 1, 2]))).astype('int')

train_df['week_low'] = ((df_train['DayOfWeek'].isin([6, 2, 3]))).astype('int')
test_df['week_low'] = ((test_df['DayOfWeek'].isin([12, 1, 2]))).astype('int')"""

In [ ]:
for col in ['DepHour']:
    train_df[col] = train_df[col].astype('object')
for col in ['DepHour']:
    test_df[col] = test_df[col].astype('object')
    
for col in ['DepTime',]:
    train_df = train_df.drop([col], axis=1)
for col in ['DepTime', ]:
    test_df= test_df.drop([col], axis=1)



In [ ]:
"""for col in ['Month', 'DayofMonth', 'DayOfWeek']:
    train_df[col] = train_df[col].apply(lambda x: x.split('-')[1]).astype(np.int32) - 1
for col in ['Month', 'DayofMonth', 'DayOfWeek']:
    test_df[col] = test_df[col].apply(lambda x: x.split('-')[1]).astype(np.int32) - 1"""

In [ ]:
train_df.head()

In [ ]:
train_df.info()

**Remember indexes of categorical features (to be passed to CatBoost)**

In [ ]:
categ_feat_idx = np.where(train_df.drop('dep_delayed_15min', axis=1).dtypes == 'object')[0]
categ_feat_idx

**Allocate a hold-out set (a.k.a. a validation set) to validate the model**

In [ ]:
X_train = train_df.drop('dep_delayed_15min', axis=1).values
y_train = train_df['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
X_test = test_df.values

In [ ]:
X_train.shape, X_test.shape, y_train.shape

In [ ]:
corrmat = train_df.drop('dep_delayed_15min', axis=1).corr()
top_corr_features = corrmat.index
plt.figure(figsize=(10,10))
#plot heat map
g=sns.heatmap(train_df.drop('dep_delayed_15min', axis=1).corr(),annot=True,cmap="RdYlGn")

In [ ]:
X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, y_train, 
                                                                test_size=.1, 
                                                                random_state=7, shuffle=True)

**Train Catboost with default arguments, passing only the indexes of categorical features.**

In [ ]:
ctb = CatBoostClassifier(task_type='GPU',loss_function='Logloss',  eval_metric='AUC',   random_seed=14, silent=True)

In [ ]:
%%time
ctb.fit(X_train_part, y_train_part,cat_features=categ_feat_idx, eval_set=(X_valid, y_valid), verbose=200);

In [ ]:
ctb_valid_pred = ctb.predict_proba(X_valid)[:, 1]

**We got some 0.756 ROC AUC on the hold-out set.**

In [ ]:
roc_auc_score(y_valid, ctb_valid_pred)

In [ ]:
fea_imp = pd.DataFrame({'imp': ctb.feature_importances_, 'col': train_df.drop('dep_delayed_15min', axis=1)})
fea_imp = fea_imp.sort_values(['imp', 'col'], ascending=[True, False]).iloc[-30:]
plt.figure(figsize=(8, 10))
sns.barplot(x='imp', y='col', data=fea_imp[1:], orient='h')
plt.title('CatBoost - Feature Importance')
plt.ylabel('Features')
plt.xlabel('Importance');

In [ ]:
%%time
ctb.fit(X_train, y_train, cat_features=categ_feat_idx);

In [ ]:
сtb_test_pred = ctb.predict_proba(X_test)[:, 1]

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    sample_sub_clf = pd.read_csv(PATH_TO_DATA / 'sample_submission.csv', 
                             index_col='id')
    sample_sub_clf['dep_delayed_15min'] = сtb_test_pred
    sample_sub_clf.to_csv('clf_pred.csv')

In [ ]:
!head clf_pred.csv

In [ ]:
from IPython.display import FileLink
FileLink('clf_pred.csv')

Now's your turn! Go and improve the model to beat **"A2 baseline (10 credits)"** - **0.75914** LB score. It's crucial to come up with some good features. 

For discussions, stick to the **#a2_kaggle_fall2019** thread in the **mlcourse_ai_news** [ODS Slack](http://opendatascience.slack.com) channel. Serhii Romanenko (@serhii_romanenko) will be there to help. 

Welcome to Kaggle!

<img src='https://habrastorage.org/webt/fs/42/ms/fs42ms0r7qsoj-da4x7yfntwrbq.jpeg' width=50%>
*from the ["Nerd Laughing Loud"](https://www.kaggle.com/general/76963) thread.*